In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# TODO

1. ✅条件を一つ指定して、そこからデータを取得する
2. ✅取得できた条件のデータから平均値のデータを取得
3. ✅すべての条件で、平均値のデータを取得
    1. ✅関数 `convertPprofTime()` において `1:16:15.802` このような入力に対する変換の対応
4. 以前と同様に予測を実施

In [2]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

# Inclusive

In [3]:
resVar: str = "Inclusive"

In [4]:
trainDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

In [5]:
testDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

In [6]:
testDF.reset_index()

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
0,0,100.0,0.00374,1.321583e+02,1.000,1.0,.TAU_application,512,256,64,1.333100e+02
1,1,100.0,1:12.675,1.321583e+02,1.000,2828.0,int_main(int_char_**),512,256,64,1.333100e+02
2,2,17.9,"14,024",2.386567e+01,256.000,67108900.0,void_CalcKinematicsForElems(Domain,512,256,64,9.332031e-02
3,3,14.6,"19,437",1.880733e+01,255.000,0.0,MPI_Allreduce(),512,256,64,7.622353e-02
4,4,7.7,433,1.027200e+01,769.000,10503.8,void_CommSend(Domain,512,256,64,1.331339e-02
...,...,...,...,...,...,...,...,...,...,...,...
859,27,0.0,0.0122,1.226667e-05,1.000,0.0,void_InitMeshDecomp(Int_t_Int_t_Int_t,1000,1024,128,1.220000e-05
860,28,0.0,0.00156,3.593333e-06,1.000,2.0,void_ParseCommandLineOptions(int_char_**_Int_t...,1000,1024,128,3.560000e-06
861,29,0.0,0.002,2.020000e-06,2.000,0.0,StrToInt,1000,1024,128,1.000000e-06
862,30,0.0,0.000458,4.553333e-07,1.000,0.0,MPI_Comm_size(),1000,1024,128,4.580000e-07


In [7]:
list_series: list[pd.Series] = []

In [8]:
averaged_rawDF_lulesh_actually: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=[729],
    list_iteration=[512],
    list_size=[96],
    list_csvDir=[
        "./csv_files/lulesh_1st/",
        "./csv_files/lulesh_2nd/",
        "./csv_files/lulesh_3rd/",
    ],
    resVar="Inclusive",
)

averaged_rawDF_lulesh_actually

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
0,100.0,0.00333,9.618643e+02,1.000000e+00,1.000000e+00,.TAU_application,729,512,96,9.610680e+02
1,100.0,10:32.533,9.618643e+02,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96,9.610680e+02
2,16.7,1:34.148,1.609003e+02,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96,3.140762e-01
3,11.4,1:49.284,1.092980e+02,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96,2.138630e-01
4,6.9,1:06.789,6.681733e+01,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96,1.471545e-07
5,3.9,"2,059",3.896433e+01,1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96,2.437411e-02
6,3.7,"35,099",3.661067e+01,1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96,2.283604e-02
7,1.6,"15,124",1.623867e+01,1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96,8.199957e-04
8,1.1,"3,072",1.103867e+01,5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96,2.069531e-02
9,0.7,626,7.194333e+00,5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96,1.319298e-02


In [9]:
sampleDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=[729],
    list_iteration=[512],
    list_size=[96],
    csvDir="./csv_files/lulesh_1st/",
)

sampleDF2_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=[729],
    list_iteration=[512],
    list_size=[96],
    csvDir="./csv_files/lulesh_2nd/",
)

sampleDF3_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=[729],
    list_iteration=[512],
    list_size=[96],
    csvDir="./csv_files/lulesh_3rd/",
)

In [10]:
sampleDF_lulesh

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,0.00333,16:01.068,1.000000e+00,1.000000e+00,.TAU_application,729,512,96
1,100.0,10:32.533,16:01.068,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96
2,16.7,1:34.148,2:40.807,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96
3,11.4,1:49.284,1:49.284,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96
4,6.9,1:06.789,1:06.789,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96
5,3.9,"2,059","37,463",1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96
6,3.7,"35,099","35,099",1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96
7,1.6,"15,124","15,124",1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96
8,1.1,"3,072","10,596",5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96
9,0.7,626,"6,768",5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96


In [11]:
# Inclusive の整形
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF_lulesh["Inclusive"]))
sampleDF_lulesh["Inclusive"] = list(_tmp_converted)
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF2_lulesh["Inclusive"]))
sampleDF2_lulesh["Inclusive"] = list(_tmp_converted)
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF3_lulesh["Inclusive"]))
sampleDF3_lulesh["Inclusive"] = list(_tmp_converted)

In [12]:
# Exclusive の整形
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF_lulesh["Exclusive"]))
sampleDF_lulesh["Exclusive"] = list(_tmp_converted)
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF2_lulesh["Exclusive"]))
sampleDF2_lulesh["Exclusive"] = list(_tmp_converted)
_tmp_converted: list[float] = map(convertPprofTime, list(sampleDF3_lulesh["Exclusive"]))
sampleDF3_lulesh["Exclusive"] = list(_tmp_converted)

In [13]:
sampleDF_lulesh

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,3.330000e-06,9.610680e+02,1.000000e+00,1.000000e+00,.TAU_application,729,512,96
1,100.0,6.325330e+02,9.610680e+02,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96
2,16.7,9.414800e+01,1.608070e+02,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96
3,11.4,1.092840e+02,1.092840e+02,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96
4,6.9,6.678900e+01,6.678900e+01,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96
5,3.9,2.059000e+00,3.746300e+01,1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96
6,3.7,3.509900e+01,3.509900e+01,1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96
7,1.6,1.512400e+01,1.512400e+01,1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96
8,1.1,3.072000e+00,1.059600e+01,5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96
9,0.7,6.260000e-01,6.768000e+00,5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96


In [14]:
sampleDF2_lulesh

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,3.390000e-06,9.621990e+02,1.000000e+00,1.000000e+00,.TAU_application,729,512,96
1,100.0,6.291300e+02,9.621990e+02,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96
2,16.7,9.431900e+01,1.610700e+02,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96
3,11.2,1.075000e+02,1.075000e+02,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96
4,7.0,6.688100e+01,6.688100e+01,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96
5,4.3,2.025000e+00,4.112500e+01,1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96
6,4.0,3.880700e+01,3.880700e+01,1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96
7,1.8,1.739500e+01,1.739500e+01,1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96
8,1.2,3.049000e+00,1.145100e+01,5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96
9,0.8,5.970000e-01,7.703000e+00,5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96


In [15]:
sampleDF3_lulesh

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,3.370000e-06,9.623260e+02,1.000000e+00,1.000000e+00,.TAU_application,729,512,96
1,100.0,6.298490e+02,9.623260e+02,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96
2,16.7,9.417200e+01,1.608240e+02,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96
3,11.5,1.111100e+02,1.111100e+02,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96
4,6.9,6.678200e+01,6.678200e+01,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96
5,4.0,2.078000e+00,3.830500e+01,1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96
6,3.7,3.592600e+01,3.592600e+01,1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96
7,1.7,1.619700e+01,1.619700e+01,1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96
8,1.2,3.046000e+00,1.106900e+01,5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96
9,0.7,6.450000e-01,7.112000e+00,5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96


In [16]:
expVar: list[str] = ["process", "iteration", "size"]
resVar: str

In [17]:
resVar = "Inclusive"
averaged_DF_lulesh_expected: pd.DataFrame = ret_averagedDF(
    inputDFs=[sampleDF_lulesh, sampleDF2_lulesh, sampleDF3_lulesh], resVar=resVar
)

In [18]:
averaged_DF_lulesh_expected

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,3.330000e-06,9.618643e+02,1.000000e+00,1.000000e+00,.TAU_application,729,512,96
1,100.0,6.325330e+02,9.618643e+02,1.000000e+00,5.644000e+03,int_main(int_char_**),729,512,96
2,16.7,9.414800e+01,1.609003e+02,5.120000e+02,4.529850e+08,void_CalcKinematicsForElems(Domain,729,512,96
3,11.4,1.092840e+02,1.092980e+02,5.110000e+02,0.000000e+00,MPI_Allreduce(),729,512,96
4,6.9,6.678900e+01,6.681733e+01,4.538700e+08,0.000000e+00,Real_t_CalcElemVolume(const,729,512,96
5,3.9,2.059000e+00,3.896433e+01,1.537000e+03,2.151800e+04,void_CommSend(Domain,729,512,96
6,3.7,3.509900e+01,3.661067e+01,1.537000e+03,0.000000e+00,MPI_Waitall(),729,512,96
7,1.6,1.512400e+01,1.623867e+01,1.844400e+04,0.000000e+00,MPI_Wait(),729,512,96
8,1.1,3.072000e+00,1.103867e+01,5.120000e+02,5.742970e+03,void_CommSyncPosVel(Domain,729,512,96
9,0.7,6.260000e-01,7.194333e+00,5.130000e+02,1.099540e+04,void_CommSBN(Domain,729,512,96


In [ ]:
%reset

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___